# 讀取 dataset

In [ ]:
import re
import numpy as np
from tqdm import tqdm
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer

# 步驟 1：讀取 movie.txt
with open('../dataset/movie_new2.txt', 'r') as f:
    lines = f.readlines()

SyntaxError: incomplete input (1985773267.py, line 8)

# 每筆觀看序列轉為電影 ID 的 list (ex: (910, 905, ...))

In [ ]:
def extract_sequence(line):
    pairs = re.findall(r'\((\d+),\s*(\d+)\)', line)
    sequence = [int(pairs[0][0])] + [int(p[1]) for p in pairs]
    return sequence

# 轉為字串形式提供給 TF-IDF (ex: "910, 905, ...")

In [ ]:
corpus = [' '.join(map(str, seq)) for seq in tqdm(sequences, desc="Building corpus...")]

# 步驟 4：使用 TF-IDF 向量化（含 N-gram）

In [ ]:
print("Building TF-IDF matrix...")
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(2, 2))  # 使用 bi-gram
X_tfidf = vectorizer.fit_transform(corpus)  # X_tfidf 是稀疏矩陣
print("finished building TF-IDF matrix.")
print("TF-IDF matrix shape:", X_tfidf.shape)

# 儲存 X_tfidf 成 .npz

In [ ]:

sparse.save_npz("X_tfidf_sparse.npz", X_tfidf)

# 查看某個使用者的非零項目

In [ ]:
user_index = 0  # 第幾位使用者
row = X_tfidf.getrow(user_index)  # 取出一行稀疏向量
nonzero_indices = row.nonzero()[1]  # 取得非零欄位的索引
feature_names = vectorizer.get_feature_names_out()

# 顯示所有非零特徵及其 TF-IDF 值
for idx in nonzero_indices:
    print(f"{feature_names[idx]}: {row[0, idx]:.4f}")